In [1]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import stats
import glob
import numpy as np

import matplotlib
import aplpy
import scipy

import scipy.ndimage as ndimage
from astropy.convolution import convolve_fft, Gaussian2DKernel, convolve
    
def std_func(values):
#     std = np.nanstd(values)
    mad = stats.mad_std(values)
    return mad

In [9]:
output_dir = './../../analysis/dendro/rms/'

inp_file = './../../data/2017.1.00793.S/2017.1.00793.S/science_goal.uid___A001_X12cc_X1c/group.uid___A001_X12cc_X1d/member.uid___A001_X12cc_X1e/product/G28-MM4_mosaic.fits'
inp_file_pb = './../../data/2017.1.00793.S/2017.1.00793.S/science_goal.uid___A001_X12cc_X1c/group.uid___A001_X12cc_X1d/member.uid___A001_X12cc_X1e/product/member.uid___A001_X12cc_X1e.G28-MM4_4_sci.spw25_27_29_31_33_35_37.mfs.IQUV.manual.pbcor.fits'

cloud = 'cloudC'

In [11]:
kr = 5 #kernal radius

print(inp_file, cloud)

hdu = fits.open(inp_file)[0]
data = hdu.data 
data = data.squeeze()

hdu = fits.open(inp_file_pb)[0]
data_pb = hdu.data[0]
data_pb = data_pb.squeeze()

header = hdu.header

rms = np.empty(data.shape) *np.nan
pb_step = 0.1
pb_levs = np.arange(0, 1, pb_step)

for pb_lev in pb_levs: 
    bool_mask = (data_pb>pb_lev) & (data_pb<=pb_lev+pb_step)
    pb_mask = np.where(bool_mask)
    data_masked = data[pb_mask]
    mad = stats.mad_std(data_masked)
    rms[bool_mask] = mad

smrms = convolve(rms, Gaussian2DKernel(kr), boundary='wrap')
smrms[~np.isfinite(rms)] = np.nan

data_new = data.copy()
data_new[data_new>3*smrms] = np.nan

rms = np.empty(data_new.shape) *np.nan
pb_step = 0.05
pb_levs = np.arange(0, 1, pb_step)

for pb_lev in pb_levs: 
    bool_mask = (data_pb>pb_lev) & (data_pb<=pb_lev+pb_step)
    pb_mask = np.where(bool_mask)
    data_masked = data_new[pb_mask]
    mad = stats.mad_std(data_masked, ignore_nan=True)
    rms[bool_mask] = mad

smrms = convolve(rms, Gaussian2DKernel(kr), boundary='wrap')
smrms[~np.isfinite(rms)] = np.nan

text = np.nanmean(rms), stats.mad_std(data_new, ignore_nan=True)
print('%0.2e %0.2e' %(text))

fits.writeto('%s/%s_rms_pbrads.fits' %(output_dir, cloud), data=smrms, header=header, overwrite=True)

./../../data/2017.1.00793.S/2017.1.00793.S/science_goal.uid___A001_X12cc_X1c/group.uid___A001_X12cc_X1d/member.uid___A001_X12cc_X1e/product/G28-MM4_mosaic.fits cloudC
9.39e-05 1.25e-04
